In [2]:
# When not doing Notebook Job
#if not 'sessions_to_rcz
#    sessions_to_run = ['Cs/SCANIMAGE/test_data_short']
#sessions_to_run = ['/home/schollab/Documents/SCANIMAGE/PTENKO_04_13_2023_fov1/acq_16']


import glob
sessions_017 = sorted(glob.glob('/mnt/md0/BRUKER/TSeries-12172024*'))
sessions_018 = sorted(glob.glob('/mnt/md0/BRUKER/TSeries-12182024*'))
sessions_to_run = sessions_017 + sessions_018
print(sessions_to_run)

['/mnt/md0/BRUKER/TSeries-12172024-1304-001', '/mnt/md0/BRUKER/TSeries-12172024-1304-002', '/mnt/md0/BRUKER/TSeries-12172024-1304-003', '/mnt/md0/BRUKER/TSeries-12172024-1304-004', '/mnt/md0/BRUKER/TSeries-12172024-1304-005', '/mnt/md0/BRUKER/TSeries-12172024-1304-006', '/mnt/md0/BRUKER/TSeries-12172024-1304-007', '/mnt/md0/BRUKER/TSeries-12172024-1304-008', '/mnt/md0/BRUKER/TSeries-12172024-1304-009', '/mnt/md0/BRUKER/TSeries-12172024-1304-010', '/mnt/md0/BRUKER/TSeries-12172024-1304-011', '/mnt/md0/BRUKER/TSeries-12172024-1304-012', '/mnt/md0/BRUKER/TSeries-12172024-1304-013', '/mnt/md0/BRUKER/TSeries-12172024-1304-014', '/mnt/md0/BRUKER/TSeries-12172024-1304-015', '/mnt/md0/BRUKER/TSeries-12172024-1304-016', '/mnt/md0/BRUKER/TSeries-12172024-1304-017', '/mnt/md0/BRUKER/TSeries-12172024-1304-018', '/mnt/md0/BRUKER/TSeries-12172024-1304-019', '/mnt/md0/BRUKER/TSeries-12172024-1304-020', '/mnt/md0/BRUKER/TSeries-12172024-1304-021', '/mnt/md0/BRUKER/TSeries-12172024-1304-022', '/mnt/md0

In [3]:
# imports and definitions. Execution below.
import os
import os.path
import cv2
import math
import h5py
import glob
import logging
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime


import code
from time import process_time
    
try:
    cv2.setNumThreads(0)
except:
    pass

try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass


import caiman as cm
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.source_extraction.cnmf import params as params
from caiman.utils.utils import download_demo
from caiman.summary_images import local_correlations_movie_offline

import tifffile

global mc

def register_one_session(parent_dir, mc_dict, keep_memmap, save_sample, sample_name):
    #pass  # For compatibility between running under Spyder and the CLI
    fnames = sorted(glob.glob(os.path.join(parent_dir, "*registered.h5")))
    #fnames = sorted(glob.glob(os.path.join(parent_dir, '*tif')))

    mc_dict['fnames'] = fnames
    mc_dict['upsample_factor_grid'] = 8 # Attempting to fix subpixel registration issue

    opts = params.CNMFParams(params_dict=mc_dict)

# %% start a cluster for parallel processing
    c, dview, n_processes = cm.cluster.setup_cluster(
        backend='local', n_processes=None, single_thread=False)
    mc = MotionCorrect(fnames, dview=dview, **opts.get_group('motion'))

    mc.motion_correct(save_movie=True)

 # Storing results in HDF5 for DeepInterpolation
    if mc_dict['pw_rigid']:
        numframes = len(mc.x_shifts_els)
    else:
        numframes = len(mc.shifts_rig)

    #os.replace(fnames[0], parent_dir + "//registered.h5")
    #datafile = h5py.File(parent_dir + '//registered.h5', 'w')
    os.replace(fnames[0], os.path.join(parent_dir, 'registered.h5'))
    datafile = h5py.File(os.path.join(parent_dir, 'registered.h5'), 'w')
    datafile.create_dataset("mov", (numframes, 512, 512))
    #datafile.create_dataset("mov", (numframes, 128, 128))
    #code.interact(local=dict(globals(), **locals())) 
    
    # Behavior changed since last stable
    #fnames_new = glob.glob(parent_dir + "//*.mmap")
    fnames_new = mc.mmap_file
    frames_written = 0
    
    for i in range(0, math.floor(numframes / 1000)):
        mov = cm.load(fnames_new[0])
        temp_data = np.array(mov[frames_written:frames_written + 1000, :, :])
        datafile["mov"][frames_written:frames_written + 1000, :, :] = temp_data
        frames_written += 1000
        del mov
    
    # handling last point
    if numframes > frames_written:
        mov = cm.load(fnames_new[0])
        temp_data = np.array(mov[frames_written:mov.shape[0], :, :])
        datafile["mov"][frames_written:mov.shape[0], :, :] = temp_data
        del mov
        
    del temp_data        
    cm.stop_server(dview=dview)

    if not keep_memmap:
        for i in range(0, len(fnames_new)):
            os.remove(fnames_new[i])

    if save_sample:
        if not os.path.isdir(os.path.join(parent_dir, "samples")):
            os.makedirs(os.path.join(parent_dir, "samples"))
        #with tifffile.TiffWriter(parent_dir+"//samples//"+sample_name, bigtiff=False, imagej=False) as tif:
        with tifffile.TiffWriter(os.path.join(parent_dir, 'samples', sample_name), bigtiff=False, imagej=False) as tif:
            for i in range(0, min(2000, numframes)):
                curfr = datafile["mov"][i,:,:].astype(np.int16)
                tif.save(curfr)
                
    datafile.close()


/tmp/ipykernel_58560/3647430063.py:26: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('load_ext autoreload')
/tmp/ipykernel_58560/3647430063.py:27: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('autoreload 2')
2024-12-19 14:17:06.464344: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-19 14:17:06.464370: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-19 14:17:06.465084: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to regist

In [4]:

def stacks_to_hdf5(parent_dir, delete_tiffs=False):    
    fnames = sorted(glob.glob(os.path.join(parent_dir, "*.tif")))
    assert len(fnames) > 0, f"No tif stacks found in {parent_dir}"
    
    mov = tifffile.imread(fnames[0])
    stack_size = mov.shape[0]
    #code.interact(local=dict(globals(), **locals())) 
    first_30 = np.flip(mov[0:30, :, :], 0)
    
    mov = cm.load(fnames[len(fnames) - 1])
    if mov.shape[0] < 30:
        # Get all shapes from last, and the remaining from second to last.
        last_30 = np.flip(mov[:, :, :], 0)
        needed = 30 - mov.shape[0]
        mov = cm.load(fnames[len(fnames) - 2])
        remainder = np.flip(mov[mov.shape[0] - needed:mov.shape[0],:,:])
        last_30 = np.append(last_30, remainder, axis=0)
    else:
        last_30 = np.flip(mov[mov.shape[0] - 30:, :, :], 0)
    
    num_frames = 60 + ((len(fnames) - 1) * stack_size) + mov.shape[0]
    
    f_out = h5py.File(os.path.join(parent_dir, "unregistered.h5"), "w")
    f_out.create_dataset("mov", (num_frames, mov.shape[1], mov.shape[2]))
    
    f_out["mov"][0:30,:,:] = first_30
    frames_written = 30
    
    for i in range(0, len(fnames)):
        mov = tifffile.imread(fnames[i])
        #code.interact(local=dict(globals(), **locals()))
        f_out["mov"][frames_written:frames_written+mov.shape[0], :, :] = np.array(mov)
        frames_written += mov.shape[0]
        del mov
        if delete_tiffs:
            os.remove(fnames[i])
        
        # Last 30
    f_out["mov"][frames_written:frames_written+30, :, :] = last_30
    f_out.close()


def register_bulk(sessions_to_run, sparse=False):
    """
    Function to run CaImAn's motion correction on prepared .h5 stack.

    Parameters:
        sessions_to_run (list): List of strings for each file directory to find data in.
        sparse (bool): True runs piecewise-nonrigid registration (NoRMCorre) after rigid registration.
                       False runs only rigid.
    Returns:
        Nothing. Does disk operations, creates new .h5 for registered calcium movie.
    """
    
    ### dataset dependent parameters
    fr = 30             # imaging rate in frames per second
    decay_time = 1      # Recommended by Ben on March 20th
    
    ### Based on Ben's recommendations
    dxy = (1.0, 1.0)
    max_shift_um = (32, 32)
    patch_motion_um = (64., 64.)
    max_shifts = [int(a/b) for a, b in zip(max_shift_um, dxy)]
    strides = tuple([int(a/b) for a, b in zip(patch_motion_um, dxy)])
    overlaps = (32, 32)
    max_deviation_rigid = 3
    #code.interact(local=locals())

    mc_dict = {
        'fr': fr,
        'decay_time': decay_time,
        'dxy': dxy,
        'pw_rigid': False,
        'max_shifts': max_shifts,
        'strides': strides,
        'overlaps': overlaps,
        'max_deviation_rigid': max_deviation_rigid,
        'border_nan': 'copy',
        'nonneg_movie': False,
        'use_cuda': False,
        'niter_rig': 5
    }

    time_deltas = []
    for i in range(0,len(sessions_to_run)):
        print(sessions_to_run[i])
        t_start = datetime.now()
        #stacks_to_hdf5(sessions_to_run[i], delete_tiffs=True)
        register_one_session(sessions_to_run[i], mc_dict, keep_memmap=False,save_sample=True, sample_name="01_rigid.tif")
        if sparse:
            mc_dict['pw_rigid'] = True
            register_one_session(sessions_to_run[i], mc_dict, keep_memmap=False, save_sample=True, sample_name="02_nonrigid.tif")


In [5]:
for acq in sessions_to_run:
    stacks_to_hdf5(acq,delete_tiffs=True)
register_bulk(sessions_to_run=sessions_to_run, sparse= True)

OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file
OME series: not an ome-tiff master file


/mnt/md0/BRUKER/TSeries-12172024-1304-001


The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)


/mnt/md0/BRUKER/TSeries-12172024-1304-002


The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
Movie average is negative. Removing 1st percentile.
Movie average is negative. Removing 1st percentile.
Movie average is negative. Removing 1st percentile.
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)


...

The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
Movie average is negative. Removing 1st percentile.
Movie average is negative. Removing 1st percentile.
Movie average is negative. Removing 1st percentile.
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)


/mnt/md0/BRUKER/TSeries-12182024-1024-030


The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
Movie average is negative. Removing 1st percentile.
Movie average is negative. Removing 1st percentile.
Movie average is negative. Removing 1st percentile.
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)


/mnt/md0/BRUKER/TSeries-12182024-1024-031


The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
In setting CNMFParams, non-pathed parameters were used; this is deprecated. In some future version of Caiman, allow_legacy will default to False (and eventually will be removed)
The local backend is an alias for the multiprocessing backend, and the alias may be removed in some future version of Caiman
Movie average is negative. Removing 1st percentile.
Movie average is negative. Removing 1st percentile.
Movie average is negative. Removing 1st percentile.
